In [211]:
from lusidtools.jupyter_tools import toggle_code

"""Futures Valuation Workflow

Attributes
----------
futures
transaction types
recipes
valuations
"""

toggle_code("Toggle Docstring")

# Computing Intraday P&L for Futures with and without Daily Close outs

In this notebook, we demonstrate how P&L can be computed for Futures instruments under two different cash accounting treatments. We look at these treatments over the course of three days for both a Coffee Futures and Orange Juice Futures contract. Note that we maintain the same market quotes and transaction amounts for each Futures contract to more easily demonstrate how the PV and P&L measures evolve over time.


## US Coffee Futures with Unrealized P&L (‘non-close out’)
In our first example, we look at Coffee futures whereby the contract's daily PV is unrealized over time. 


## US Orange Juice Futures with Realized P&L ('close out’)
In our second example, we look at an Orange Juice future where the previous days P&L is realized at the beginning of each day and booked down into a seperate cash line item. We then reset the cost basis of the transaction each day such that the computed PV at the start of each day is 0.

In [246]:
# Import generic non-LUSID packages
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import pytz
import time
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as models

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Create API client
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(), app_name="futuresLoader"
)

In [247]:
# LUSID Variable Definitions
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lusid.api.AggregationApi)

In [248]:
# Define scopes
scope = "ibor"
quotes_scope = "ibor"

# 1. Create Portfolio

In [249]:
portfolio_code = "FuturesPortfolioForPnLCalc"

try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id FuturesPortfolioForPnLCalc because it already exists in scope ibor.


# 2. Create Futures

In [216]:
# Define function that creates futures
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=models.ExoticInstrument(
            instrument_format=models.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={"ClientInternal": models.InstrumentIdValue(fut_identifier)},
        definition=futuredef,
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(futLuid)

## 2.1 Create US Coffee Future Contract

In [250]:
start_date = datetime(2021, 1, 1, tzinfo=pytz.utc)
effectiveAt = datetime(2021, 1, 1, tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 31, tzinfo=pytz.utc)

dom_ccy = "USD"
convention = "ActualActual"
contract_code = "KC"
contract_month = "H"
contract_size = 10000
country_id = "US"
fut_name = "Coffee Futures"
exchange_code = "ICE"
exchange_name = "ICE"
ticker_step = 0.01
unit_value = 1
ref_spot_price_val = 100
identifier = "FUT_ICEKCZDEC21"

# Create Futures Contract function call 2 (Coffee 'C')
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D64


## 2.2 Create Orange Juice Future Contract

In [218]:
# Variable definitions for use in create_futures_contract function call 1 (FCOJ-A)
start_date = datetime(2021, 1, 1, tzinfo=pytz.utc)
effectiveAt = datetime(2021, 1, 1, tzinfo=pytz.utc)
maturity_date = datetime(2021, 12, 31, tzinfo=pytz.utc)

dom_ccy = "USD"
convention = "ActualActual"
contract_code = "OJ"
contract_month = "H"
contract_size = 1
country_id = "US"
fut_name = "Orange Juice Futures"
exchange_code = "ICE"
exchange_name = "ICE"
ticker_step = 0.01
unit_value = 1
ref_spot_price_val = 0
identifier = "FUT_ICEOJZDEC21"


# Create Futures Contract function call 1 (FCOJ-A)
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
)

LUID_00003D66


# 3. Transactions

## 3.1 Create Transaction Type

In [251]:
new_transaction_config = [
    models.TransactionConfigurationDataRequest(
        aliases=[
            models.TransactionConfigurationTypeAlias(
                type="OpenContract",
                description="An FuturesTxnType transaction type",
                transaction_class="default",
                transaction_group="default",
                transaction_roles="Longer",
            )
        ],
        movements=[
            models.TransactionConfigurationMovementDataRequest(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
            )
        ],
        properties=None,
    )
]

new_txn_config = upsert_transaction_type_alias(
    api_factory, new_transaction_config=new_transaction_config
)

## 3.2 Book Transactions

In [252]:
# Read in transaction data read from file
futures_transactions = pd.read_excel("data/futures_data.xlsx", sheet_name="transactions")
# Output transaction data
futures_transactions

,txn_id,txn_type,trade_date,quantity,client_id,currency_id,price,tc,contract_size,portfolio
0,futa_txn_001,OpenContract,2021-01-01T08:00:00Z,10,FUT_ICEKCZDEC21,USD,100.00,10000000,"10,000.00",FuturesPortfolioForPnLCalc
1,futa_txn_002,OpenContract,2021-01-02T14:15:00Z,5,FUT_ICEKCZDEC21,USD,99.50,5000000,"10,000.00",FuturesPortfolioForPnLCalc
2,futb_txn_001,OpenContract,2021-01-01T08:00:00Z,10,FUT_ICEOJZDEC21,USD,100.00,10000000,"10,000.00",FuturesPortfolioForPnLCalc
3,crdcb_txn_001,StockIn,2021-01-02T08:00:00Z,0,FUT_ICEOJZDEC21,USD,0.00,50000,"10,000.00",FuturesPortfolioForPnLCalc
4,cshb_txn_001,FundsIn,2021-01-02T08:00:00Z,50000,NaN,USD,1.00,50000,nan,FuturesPortfolioForPnLCalc
5,futb_txn_002,OpenContract,2021-01-02T14:15:00Z,5,FUT_ICEOJZDEC21,USD,99.50,4975000,"10,000.00",FuturesPortfolioForPnLCalc
6,crdcb_txn_002,StockOut,2021-01-03T08:00:00Z,0,FUT_ICEOJZDEC21,USD,0.00,325000,"10,000.00",FuturesPortfolioForPnLCalc
7,cshb_txn_002,FundsIn,2021-01-03T08:00:00Z,-325000,NaN,USD,1.00,325000,nan,FuturesPortfolioForPnLCalc


In [236]:
for index, row in futures_transactions.iterrows():

    primary_instrument_identifier = { "Instrument/default/ClientInternal": row["client_id"] }

    if row["txn_type"] == "FundsIn":
        primary_instrument_identifier = { "Instrument/default/Currency": row["currency_id"] }

    upsert_transactions = transaction_portfolio_api.upsert_transactions(
        scope=scope,
        code=row['portfolio'],
        transaction_request=[
            models.TransactionRequest(
                transaction_id=row["txn_id"],
                type=row["txn_type"],
                instrument_identifiers=primary_instrument_identifier,
                transaction_date=row["trade_date"],
                settlement_date=row["trade_date"],
                units=row["quantity"],
                transaction_price=models.TransactionPrice(
                    price=row["price"], type="Price"
                ),
                total_consideration=models.CurrencyAndAmount(
                    amount=row["tc"], currency=row["currency_id"]
                ),
            )
        ],
    )

# 4. Quotes

## 4.1 Book Quotes

In [253]:
# Read in futures data from file
futures_prices = pd.read_excel("data/futures_data.xlsx", sheet_name="prices")
# Output futures data
futures_prices

,date,prices,luid,id_type,currency,scaling_factor,note
0,2021-01-01T08:00:00Z,100.00,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 1 opening price
1,2021-01-01T12:00:00Z,100.50,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 1 intraday MTM price
2,2021-01-01T17:00:00Z,100.50,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 1 closing price
3,2021-01-02T08:00:00Z,100.50,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 2 opening price
4,2021-01-02T14:15:00Z,99.50,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 2 intraday trade price
5,2021-01-02T17:00:00Z,98.00,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 2 closing price
6,2021-01-03T08:00:00Z,98.00,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 3 opening price
7,2021-01-03T14:15:00Z,104.00,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 3 intraday trade price
8,2021-01-03T17:00:00Z,104.00,FUT_ICEKCZDEC21,ClientInternal,USD,1.00,Day 3 closing price
9,2021-01-01T08:00:00Z,100.00,FUT_ICEOJZDEC21,ClientInternal,USD,0.00,Day 1 opening price


In [254]:
for index, row in futures_prices.iterrows():

    instrument_quotes = {
        "upsert_request_1": models.UpsertQuoteRequest(
            quote_id=models.QuoteId(
                quote_series_id=models.QuoteSeriesId(
                    provider="Lusid",
                    instrument_id=row["luid"],
                    instrument_id_type="ClientInternal",
                    quote_type="Price",
                    field="mid",
                ),
                effective_at=row["date"],
            ),
            metric_value=models.MetricValue(value=row["prices"], unit=row["currency"]),
            scale_factor=row["scaling_factor"]            
        )
    }

    # Upsert the quotes into LUSID
    response = quotes_api.upsert_quotes(
        scope=quotes_scope, request_body=instrument_quotes
    )

# 5. Valuations

## 5.1 Create valuation recipe

In [255]:
# Create a recipe to perform a valuation
configuration_recipe = models.ConfigurationRecipe(
    scope=scope,
    code="futuresValuation",
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Equity.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="0D.5D",
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="Lusid", credit="Lusid", equity="Lusid", fx="Lusid", rates="Lusid"
        ),
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=models.PricingContext(
        model_rules=[
            models.VendorModelRule(
                supplier="Lusid",
                model_name="ConstantTimeValueOfMoney",
                instrument_type="Future",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

## 5.2 Create daily valuation function

The get_daily_fut_val() function when given a date & time along with a portfolio returns a valuation of the held instruments with an indicator of their price relative to the agreed future price. This valuation therefore enables you to know if your futures are worth more or less than you agreed to pay for them at any given time.

In [225]:
def get_daily_fut_val(date, portfolio_code):

    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(scope=scope, code="futuresValuation"),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            models.AggregateSpec("Instrument/Definition/ContractSize", "Value"),
            models.AggregateSpec("Quotes/Price", "Value"),
            models.AggregateSpec("Holding/default/Units", "Value"),
            models.AggregateSpec("Valuation/PV/Amount", "Value"),
            models.AggregateSpec("Valuation/Exposure/Amount", "Value"),
            models.AggregateSpec("Valuation/PnL/Tm1", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Value",
            "Valuation/Exposure/Amount": "Exposure",
            "Valuation/PnL/Tm1": "PnL (1-day)",
        },
        inplace=True,
    )

    return vals_df

## 5.3 Daily valuations

### Day 1

At 8:00am on Day 1, we enter into both a Coffee Futures and Orange Juice Futures contract. At 12:00pm noon, we decide to run an intraday mark to market valuation. We can see the contract price for both instruments has moved from the original purchase price of 100.00 to 100.50. This gives us an intraday PV for both contracts of 50,000 USD.

At the end of the day, we see the contract prices remain at 100.50 yielding us the same valuation.

In [256]:
get_daily_fut_val("2021-01-01T08:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Aggregation/Errors
0,0.00,"10,000,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",100.00,10.00,None,[Failed to evaluate scripted task. Result was ...
1,0.00,"10,000,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,100.00,10.00,None,[Failed to evaluate scripted task. Result was ...


In [257]:
get_daily_fut_val("2021-01-01T12:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Aggregation/Errors
0,"50,000.00","10,050,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",100.50,10.00,None,[Failed to evaluate scripted task. Result was ...
1,"50,000.00","10,050,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,100.50,10.00,None,[Failed to evaluate scripted task. Result was ...


In [258]:
get_daily_fut_val("2021-01-01T17:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day),Aggregation/Errors
0,"50,000.00","10,050,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",100.50,10.00,None,[Failed to evaluate scripted task. Result was ...
1,"50,000.00","10,050,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,100.50,10.00,None,[Failed to evaluate scripted task. Result was ...


### Day 2

At 8:00am on Day 2, we again run a start of day valuation of the two contracts. We can see the Coffee Futures contract still has a PV of 50,000 USD while the Orange Juice contract has a PV of 0. The reason for this discrepency is that we've booked a cost basis adjustment for the Orange Juice Futures contract which moved it's PV back to 0. This cost basis adjustment was made possible through a 'StockIn' transaction where we set both the quantity and price to 0 and the total commitment (TC) to that of the previous day's P&L. 

At 2:15pm, we now book a new transaction in the amount of 5 contracts for each commodity. We can see the intraday PV of the Coffee Futures is now -75,000 USD while the PV of the Orange Juice Futures is now -100,000 USD. Note that Coffee Futures are still using the reference spot price as the cost basis.

The PV's of the two futures are computed as follows:                      

#### Coffee Futures (PV) = (99.50 - 100.0) * 15 * 10,000 =  -75,000 USD  
Where 100 is the reference spot price in the contract definition

#### Orange Juice Futures (PV) = (99.50 - 100.5) * 10 * 10,000 + (99.5 - 99.5) * 5 * 10,000 = -100,000 USD
Where 100.5 is the EOD price from the previous day and hence the cost basis
<br/>

At the close of day, both futures prices move down further to 98.0 Our PVs will now be computed as follows:

#### Coffee Futures (PV) = (98.0 - 100.0) * 15 * 10,000 = -300,000 USD
Where 100 is the reference spot price in the contract definition
<br/>
#### Orange Juice Futures (PV) = (98.0 - 100.5) * 10 * 10,000 + (98.0 -  99.5) *  5 * 10,000 = -325,000 USD 
Where 100.5 and 99.5 are the current cost basis of the two transactions.



In [259]:
get_daily_fut_val("2021-01-02T08:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day)
0,"50,000.00","10,050,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",100.50,10.00,"50,000.00"
1,0.00,"10,050,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,100.50,10.00,"50,000.00"
2,"50,000.00","50,000.00",USD,None,1.00,nan,"50,000.00",0.00


In [260]:
get_daily_fut_val("2021-01-02T14:15:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day)
0,"-75,000.00","14,925,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",99.50,15.00,"-150,000.00"
1,"-100,000.00","14,925,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,99.50,15.00,"-150,000.00"
2,"50,000.00","50,000.00",USD,None,1.00,nan,"50,000.00",0.00


In [261]:
get_daily_fut_val("2021-01-02T17:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day)
0,"-300,000.00","14,700,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",98.00,15.00,"-375,000.00"
1,"-325,000.00","14,700,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,98.00,15.00,"-375,000.00"
2,"50,000.00","50,000.00",USD,None,1.00,nan,"50,000.00",0.00


### Day 3

At 8:00am on Day 3, we again take a start of day valuation of the two contracts. We can see the Coffee contract has a PV of 300,000 USD (using the same price as previous days close) while the Orange Juice contracts PV has reset again with a value of 0 USD. We've updated today's cash by subtracting yesterday's PV of -325,000 USD from yesterdays cash balance of 50,000 USD to arrive at today's new cash balance of -275,000 USD
<br />
We then take an intraday and end of day valuation as follows:


#### Coffee Futures (PV) = (104.0 - 100.0) * 15 * 10,000 = 600,000 USD  
Where 100 is the reference spot price in the contract definition

#### Orange Juice Futures (PV) = (104.0 - 98.0) * 10 * 10,000 +  (104.0 -  98.0) *  5 * 10,000 = 900,000 USD
Where 98.0 is the current cost basis of the two corresponding transactions. 


In [262]:
get_daily_fut_val("2021-01-03T08:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day)
0,"-300,000.00","14,700,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",98.00,15.00,"-375,000.00"
1,0.00,"14,700,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,98.00,15.00,"-375,000.00"
2,"-275,000.00","-275,000.00",USD,None,1.00,nan,"-275,000.00",0.00


In [263]:
get_daily_fut_val("2021-01-03T14:15:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day)
0,"600,000.00","15,600,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",104.00,15.00,"675,000.00"
1,"900,000.00","15,600,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,104.00,15.00,"675,000.00"
2,"-275,000.00","-275,000.00",USD,None,1.00,nan,"-275,000.00",0.00


In [264]:
get_daily_fut_val("2021-01-03T17:00:00Z", portfolio_code)

,Value,Exposure,InstrumentName,ClientInternal,Instrument/Definition/ContractSize,Quotes/Price,Holding/default/Units,PnL (1-day)
0,"600,000.00","15,600,000.00",Coffee Futures,FUT_ICEKCZDEC21,"10,000.00",104.00,15.00,"900,000.00"
1,"900,000.00","15,600,000.00",Orange Juice Futures,FUT_ICEOJZDEC21,1.00,104.00,15.00,"900,000.00"
2,"-275,000.00","-275,000.00",USD,None,1.00,nan,"-275,000.00",0.00
